In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# GÖREV 1 : Veri Hazırlama işlemlerini gerçekleştiriniz.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 20)


In [ ]:
def create_user_movie_df():
    movie = pd.read_csv('../input/movielens-20m-dataset/movie.csv')
    rating = pd.read_csv('../input/movielens-20m-dataset/rating.csv')
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

In [ ]:
user_movie_df = create_user_movie_df()
user_movie_df.head()

In [ ]:
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=45).values)
#rastgele kullanıcı getirir

In [ ]:
# GÖREV 2 : Öneri yapılacak kullanıcının izlediği filmleri belirleyiniz

In [ ]:
random_user_df = user_movie_df[user_movie_df.index == random_user]

In [ ]:
random_user_df.head()
#rastgele kullanınıcı ve tüm filmler

In [ ]:
#rastgele secilen kullanıcının izlediği filmler
#random_user_df de boş değerleri getirme
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()

In [ ]:
# GÖREV 3 : Aynı filmleri izleyen diğer kullanıcıların verisine ve Id'lerine erişiniz

In [ ]:
movies_watched_df = user_movie_df[movies_watched]
movies_watched_df.head()
#Diğer kullanıcıların sectigimiz kullanıcının izledigi filmlere verdikleri puanlar

In [ ]:
user_movie_count = movies_watched_df.T.notnull().sum()
#herkesin izledigi film sayısı
user_movie_count = user_movie_count.reset_index()

user_movie_count.columns = ["userId", "movie_count"]



In [ ]:
user_movie_count.head()

In [ ]:
user_movie_count[user_movie_count["movie_count"] > 20].sort_values("movie_count", ascending=False)
#o kisiyle daha cok ortak film izleyenleri getirdik

In [ ]:
users_same_movies = user_movie_count[user_movie_count["movie_count"] > 20]["userId"]

In [ ]:
users_same_movies.head()

In [ ]:
users_same_movies.count()

In [ ]:
users_same_movies.index

In [ ]:
# GÖREV 4 : Öneri yapılacak kullanıcı ile en benzer kullanıcıları belirleyiniz

In [ ]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])

final_df.head()

# sütunda random userın izlediği filmler ve satırda aynı filmleri izleyen kullanıcı id leri var
# değer olarak bu kullanıcıların random userın izledigi filmlere verdikleri puanlar var

In [ ]:
final_df.T.corr()

#kullanıcıların birbiri ile korelasyonları

In [ ]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
# corr_df kullanıcıların birbiri ile korelasyonlarını verir
# birden fazla aynı değerleri temizledik
'''
userId    userId 
28866.0   67756.0   -0.936065
60562.0   37121.0   -0.915003
34103.0   80593.0   -0.898718
62575.0   21398.0   -0.896612
117826.0  48416.0   -0.883969
'''

In [ ]:
corr_df = pd.DataFrame(corr_df, columns=["corr"])
#corr_df i dataframe e cevirdik

corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
#corr üst seviyede kalıyor bunu düzelttik

corr_df.head()

In [ ]:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
    ["user_id_2", "corr"]].reset_index(drop=True)
#random user ile diğer kullanıcılar arasında korelasyon değeri lazım
#bunun için user1 i random user a esitledik
'''
user_id_2   	corr
0	132307.0	0.660726
1	42696.0	   0.661268
2	1362.0	   0.665461
3	124546.0	0.669585
4	73844.0	0.670105

'''

top_users = top_users.sort_values(by='corr', ascending=False)

top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

top_users

'''
    userId	       corr
27	28941.0	  1.000000
26	45158.0	  0.800749
25	101628.0	0.790405
24	127259.0	0.763925
'''


In [ ]:
rating = pd.read_csv('../input/movielens-20m-dataset/rating.csv')
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')

top_users_ratings.head(50)

In [ ]:
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]
#kendi kullanıcımız olmasın

In [ ]:
# GÖREV 5 : Weighted Average Recommendation Score'u hesaplayınız ve ilk 5 filmi tutunuz

In [ ]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.head()

In [ ]:
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index() #movie id alt seviyede kalıyor
recommendation_df.head()

In [ ]:
# GÖREV 6 : Kullanıcının izlediği filmlerden en son en yüksek puan verdiği filmin adına göre item-based öneri yapınız

In [ ]:
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending=False)[0:5]

In [ ]:
#user based öneri 
movie = pd.read_csv('../input/movielens-20m-dataset/movie.csv')
movies_to_be_recommend.merge(movie[["movieId", "title"]])

In [ ]:
user_id = random_user
movie_id=rating[(rating["userId"]==user_id)&(rating["rating"]==5.0)].\
  sort_values(by="timestamp",ascending=False)["movieId"][0:1].values[0]
#movie_id : 7

In [ ]:
movie[movie["movieId"]==7]["title"]
#movie: Sabrina (1995)

In [ ]:
#ıtem based e göre öneri

def item_based_recommender(movie_name, user_movie_df):
    movie_name = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(6)

In [ ]:
movies_from_item_based = item_based_recommender(movie[movie["movieId"] == 7]["title"].values[0], user_movie_df)

In [ ]:
movies_from_item_based[1:6].index

'''
['Intouchables (2011)', 'Father of the Bride (1991)',
       'Anna and the King (1999)', 'Runaway Bride (1999)',
       'Phantom of the Opera, The (2004)'],
'''